In [ ]:
import os
import json
from dadk.BinPol import *

with_error = True
while with_error:
    print()
    with_error = False

    for N in range(65):

        src_filename = f'results-N={N}.json'
        tgt_filename = f'results-N={N}-short.json' 

        if not os.path.isfile(src_filename):
            continue
    
        if (not os.path.isfile(tgt_filename)) or (os.path.getmtime(src_filename) > os.path.getmtime(tgt_filename)):
    
            print(f'N {N:5d} ... ', end='') 
            
            var_shape_set = VarShapeSet(BitArrayShape('x', (N, N), one_hot=OneHot.two_way))
            penalty = BinPol(var_shape_set)
            for t in range(N):
                penalty.add_exactly_1_bit_on(bits=[('x', t, c) for c in range(N)])
            for c in range(N):
                penalty.add_exactly_1_bit_on(bits=[('x', t, c) for t in range(N)])
        
            try:
                with open(src_filename) as fp:
                    data = json.load(fp)
                for M in sorted(data.keys()):
                    for seed_id in sorted(data[M].keys()):
                        if '_data' in seed_id:
                            continue
                        data[M][seed_id] = [ (entry[0], int(M)*penalty.compute(entry[2])) for entry in data[M][seed_id] ]
    
                with open(tgt_filename, 'w') as fp:
                    json.dump(data, fp)
                print('done')    
                
            except Exception as oops:
                print('error', oops)
                with_error = True
        else:
            print(f'N {N:5d} ... skipped (no update needed)')

print('\nready')

In [ ]:
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from tabulate import tabulate
import json
import os
import math
import numpy as np
from typing import List           

html = []

html.append('<html>')
html.append('<body>')
html.append('<table>')
html.append(f'\t<tr>')
html.append(f'\t\t<td></td>')

no = 1

for N in range(65):
    
    filename = os.path.join('.', f'results-N={N}-short.json' )
    if not os.path.isfile(filename):
        continue        

    coordinates = [(1_000_000 * math.cos((index / N) * 2 * math.pi), 1_000_000 * math.sin((index / N) * 2 * math.pi)) for index in range(N)]
    
    distance = 0
    for i in range(N):
        distance += int(math.sqrt((coordinates[(i+1)%N][0] - coordinates[i][0]) ** 2 + (coordinates[(i+1)%N][1] - coordinates[i][1]) ** 2))
    print(f'distance {distance:,d}')    
    

    with open(filename, 'r') as fp:
        summary = json.load(fp)  
        objective = summary.get('objective', [])
        summary = {int(M):{seed:summary[M][seed] for seed in summary[M]} for M in summary}

    seed_ids = []
    for M in summary:
        for seed_id in summary[M]:
            if '_data' not in seed_id:
                if seed_id not in seed_ids:
                    seed_ids.append(seed_id)
    seed_ids = sorted(list(set(seed_ids)))

    print(seed_ids)

    max_mean_value = -1
    for M in summary:
        
        values = []
        counter = 0
        for seed in seed_ids:
            if seed in summary[M]:
                values += [entry[0] / distance for entry in summary[M][seed] if entry[1] == 0]
                counter += len(summary[M][seed])
        
        if counter > 0:
            rate = 100*len(values)/counter
            if rate >= 99.5:
                if len(values)>0:
                    mean_value = np.mean(values)
                    if mean_value > max_mean_value:
                        max_mean_value = mean_value

        
    fig, ax = plt.subplots(1, 1, figsize=(12,4)  )
    ax2 = ax.twinx()
    fig.canvas.header_visible = False

    last_M_0 = None
    first_M_100 = None
    
    for M in summary:
        values = []
        counter = 0
        for seed in seed_ids:
            if seed in summary[M]:
                values += [entry[0] / distance for entry in summary[M][seed] if entry[1] == 0]
                counter += len(summary[M][seed])
                
        if counter > 0:
            rate = 100*len(values)/counter
            ax.scatter( [M], [rate], c='blue', s=10)
        else:
            rate = 0

        if rate == 0:
            last_M_0 = M
        if rate > 99.99 and first_M_100 is None:
            first_M_100 = M                
       
        if len(values)>0:
            ax2.scatter( [M], np.min(values), c='green', s=10)
            #ax2.scatter( [M], np.max(values), c='black', s=10)
            ax2.scatter( [M], np.mean(values), c='red', s=10)   

    print(last_M_0, first_M_100)
    
    epsilon = 0.05
    
    ax2.set_ylabel("rel. mean energy",color="red",fontsize=14)

    ylim2 = (1.0, 1.2*max_mean_value)
    delta2 = epsilon*(ylim2[1]-ylim2[0])
    ax2.set_ylim((ylim2[0] - delta2, ylim2[1] + delta2))        
    ax.set_ylabel("% valid solutions",color="blue",fontsize=14)
    ylim1 = (0.0, 100.0)
    delta1 = epsilon*(ylim1[1]-ylim1[0])
    ax.set_ylim((ylim1[0] - delta1, ylim1[1] + delta1))
    ax.set_xlabel(f"factor for penalty [{min(summary.keys()):,d} - {max(summary.keys()):,d}]", color="black",fontsize=14)
    #plt.xlim((min(summary.keys()), max(summary.keys())))
    plt.title(f"{N:,d} cities, {N*N:,d} bits, {(N*N)*(N*N):,d} iterations")
    png_filename = filename.replace('.json', '.png')
    plt.savefig(png_filename, dpi=150)
    plt.show()
    plt.close()

    html.append(f'\t\t<td><h1>{N}<a href="{png_filename}"><img src="{png_filename}" width="33%" height="33%"/></a></h1></td>')

    no +=1
    if no % 4 == 0:
        html.append(f'\t<tr>')
        html.append(f'\t</tr>')

html.append('</table>')
html.append('</body>')
html.append('</html>')

with open('summary.html', 'w') as fp:
    fp.write('\n'.join(html))